# Goal

Our aim was to change the structure and the parameters of the standard Vogels and Abbott (VA) Network, presented [here](https://github.com/albertoarturovergani/CNT-2024/blob/3d93c0d922ef2b6a57b0e4fde789f5d98b557733/notebooks/paper_balance-network.ipynb), and to understand how these changes impacted the results and behavior of the model.

(NOTE: baseline code for functions etc. was taken from the source notebooks seen in the labs, contained in [this](https://github.com/albertoarturovergani/CNT-2024/tree/main/notebooks) GitHub repository)

----------------------------------------------------------------------

In [1]:
fileName='VA_balance-network'

In [2]:
import pyNN.spiNNaker as sim
from pyNN import space
import numpy as np
import matplotlib.pyplot as plt
import numpy as np
import matplotlib.pyplot as plt
import scipy
import pickle
import time
import datetime
import json
import warnings
warnings.filterwarnings('ignore')

savePath = f'./outputs/' # remember to create the folder if not already present (mkdir ./notebooks/outputs)
dt_string = datetime.datetime.today().isoformat() # ISO8601 ! :-)
tag = dt_string
saveName = f'{savePath}{fileName}-{tag}'
print(saveName)
PARS={}


./outputs/VA_balance-network-2024-02-07T11:04:53.749289


# step0: import the simulator

In [3]:
import socket
try:
    import pyNN.spiNNaker as sim
except ModuleNotFoundError:
    import pyNN.brian2 as sim

from pyNN.random import NumpyRNG, RandomDistribution
from pyNN.utility.plotting import Figure, Panel
import matplotlib.pyplot as plt
import matplotlib.colors as colors
import time
from pyNN import space

# step1: setup the simulator

In [4]:
dt = 1          # (ms) simulation timestep
tstop = 1000      # (ms) simulaton duration
delay = 2 # (ms)

sim.setup(
    timestep=dt,
    min_delay=delay,
    max_delay= dt * 144) # [ms] # not that the max_delay supported by SpiNNaker is timestep * 144

rngseed = 98766987
rng = NumpyRNG(seed=rngseed, parallel_safe=True)

2024-02-07 11:04:53 INFO: Read cfg files: /home/bbpnrsoa/sPyNNaker/lib/python3.8/site-packages/spinn_front_end_common/interface/spinnaker.cfg, /home/bbpnrsoa/sPyNNaker/lib/python3.8/site-packages/spynnaker/pyNN/spynnaker.cfg, /home/bbpnrsoa/.spynnaker.cfg
2024-02-07 11:04:53 INFO: Will search these locations for binaries: /home/bbpnrsoa/sPyNNaker/lib/python3.8/site-packages/spinn_front_end_common/common_model_binaries : /home/bbpnrsoa/sPyNNaker/lib/python3.8/site-packages/spynnaker/pyNN/model_binaries
2024-02-07 11:04:53 WARNING: /home/bbpnrsoa/VA_neuronal_model/notebooks/reports has 3 old reports that have not been closed
2024-02-07 11:04:53 INFO: Setting time scale factor to 1.
2024-02-07 11:04:53 INFO: Setting machine time step to 1000 micro-seconds.


['/home/bbpnrsoa/sPyNNaker/lib/python3.8/site-packages/spinn_front_end_common/interface/spinnaker.cfg', '/home/bbpnrsoa/sPyNNaker/lib/python3.8/site-packages/spynnaker/pyNN/spynnaker.cfg', '/home/bbpnrsoa/.spynnaker.cfg']


# step2: decide the cell types and parameters

In [5]:
# cell type
celltype = sim.IF_cond_exp
#  Leaky integrate and fire model with fixed threshold and exponentially-decaying post-synaptic conductance.
#  http://neuralensemble.org/docs/PyNN/_modules/pyNN/standardmodels/cells.html#IF_cond_exp

# Cell parameters
area = 20000.     # (µm²)
tau_m = 20.       # (ms)
cm = 1.           # (µF/cm²)
g_leak = 5e-5     # (S/cm²)

E_leak = None
E_leak = -60.  # (mV)
v_thresh = -50.   # (mV)
v_reset = -60.    # (mV)
t_refrac = 5.     # (ms) (clamped at v_reset)
v_mean = -60.     # (mV) mean membrane potential, for calculating CUBA weights
tau_exc = 5.      # (ms)
tau_inh = 10.     # (ms)
Erev_exc = 0.     # (mV)
Erev_inh = -80.   # (mV)

# === Calculate derived parameters ===
area = area * 1e-8                     # convert to cm²
cm = cm * area * 1000                  # convert to nF
Rm = 1e-6 / (g_leak * area)            # membrane resistance in MΩ
assert tau_m == cm * Rm                # just to check

cell_params = {'tau_m': tau_m,
               'tau_syn_E': tau_exc,
               'tau_syn_I': tau_inh,
               'v_rest': E_leak,
               'v_reset': v_reset,
               'v_thresh': v_thresh,
               'cm': cm,
               'tau_refrac': t_refrac,
               'i_offset': 0
               }

cell_params['e_rev_E'] = Erev_exc
cell_params['e_rev_I'] = Erev_inh

print(cell_params)

{'tau_m': 20.0, 'tau_syn_E': 5.0, 'tau_syn_I': 10.0, 'v_rest': -60.0, 'v_reset': -60.0, 'v_thresh': -50.0, 'cm': 0.2, 'tau_refrac': 5.0, 'i_offset': 0, 'e_rev_E': 0.0, 'e_rev_I': -80.0}


# step3: making cell populations


In [6]:
r_ei_param={'1': 1.0, '2': 2.0, '3': 3.0, 'default': 4.0, '5': 5.0, '6': 7.0, '7': 9.0, '8': 10.0}
pconn_param = {'1': 0.005, '2': 0.01, 'default': 0.02, '4': 0.03}
Gexc_param = {'default': 4.}
Ginh_param = {'default': 51.}
stim_dur_param = {'1': 5,'2': 30,'default': 50., '4': 70,'5': 100}
rate_param = {'1': 20,'2': 50,'default': 100,'4': 150,'5': 200}
param = {'r_ei': r_ei_param, 'pconn': pconn_param, 'Gexc': Gexc_param, 'Ginh': Ginh_param, 'stim_dur':stim_dur_param, 'rate': rate_param}

In [7]:
class simulations:
    def __init__(self, sim, celltype, cell_params, rng, param):
        self.sim = sim
        self.celltype = celltype
        self.cell_params = cell_params
        self.rng = rng
        self.param = param
        self.pops = {}
        self.stateVarsSet=[]
        self.resultsSet=[]

    def create_populations(self,r_ei):
        n = 1500         # number of cells
        rng = self.rng
        cell_params = self.cell_params
        celltype = self.celltype
        n_exc = int(round((n * r_ei / (1 + r_ei))))  # number of excitatory cells
        n_inh = n - n_exc                            # number of inhibitory cells
        print("{} {}".format(n_exc, n_inh))

        self.pops = {
            'exc': sim.Population(
                                        n_exc,
                                        celltype(**cell_params),
                                        label="Excitatory_Cells"),

            'inh': sim.Population(
                                        n_inh,
                                        celltype(**cell_params),
                                        label="Inhibitory_Cells")
        }

        self.pops['exc'].record(["spikes", 'v', 'gsyn_exc', 'gsyn_inh'])
        self.pops['inh'].record(["spikes", 'v', 'gsyn_exc', 'gsyn_inh'])


        uniformDistr = RandomDistribution('uniform', [v_reset, v_thresh], rng=rng)
        self.pops['exc'].initialize(v=uniformDistr)
        self.pops['inh'].initialize(v=uniformDistr)

        #sim.set_number_of_neurons_per_core(sim.IF_cond_exp, 50)      # this will set
        # 50 neurons per core
        self.pops.keys()
        

    def create_connections(self, pconn, Gexc, Ginh, stim_dur, rate):
        w_exc = Gexc * 1e-3              # We convert conductances to uS
        w_inh = Ginh * 1e-3
        print(Gexc, Ginh)
        print(w_exc, w_inh)
        print(pconn)
        pops = self.pops
        exc_synapses = sim.StaticSynapse(weight=w_exc, delay=delay)
        inh_synapses = sim.StaticSynapse(weight=w_inh, delay=delay)

        exc_conn = sim.FixedProbabilityConnector(pconn)
        inh_conn = sim.FixedProbabilityConnector(pconn)
        connections = {

            'e2e': sim.Projection(
                pops['exc'],
                pops['exc'],
                exc_conn,
                receptor_type='excitatory',
                synapse_type=exc_synapses),

            'e2i': sim.Projection(
                pops['exc'],
                pops['inh'],
                exc_conn,
                receptor_type='excitatory',
                synapse_type=exc_synapses),

            'i2e': sim.Projection(
                pops['inh'],
                pops['exc'],
            inh_conn,
            receptor_type='inhibitory',
            synapse_type=inh_synapses),

            'i2i': sim.Projection(
                pops['inh'],
                pops['inh'],
                inh_conn,
                receptor_type='inhibitory',
                synapse_type=inh_synapses)

                }

        connections.keys()
        n_thalamic_cells = 20
        exc_conn = None

        pops['thalamus'] = sim.Population(
            n_thalamic_cells,
            sim.SpikeSourcePoisson(rate=rate, duration=stim_dur),
            label="expoisson")
        pops['thalamus'].record("spikes")


        rconn = 0.01
        ext_conn = sim.FixedProbabilityConnector(rconn)

        connections['ext2e'] = sim.Projection(
            pops['thalamus'],
            pops['exc'],
            ext_conn,
            receptor_type='excitatory',
            synapse_type=sim.StaticSynapse(weight=0.1))

        connections['ext2i'] = sim.Projection(
            pops['thalamus'],
            pops['inh'],
            ext_conn,
            receptor_type='excitatory',
            synapse_type=sim.StaticSynapse(weight=0.1))

        connections.keys(), pops.keys()
        

    def run_simulation(self):
        tstop = 1000 # (ms)
        # simulation run
        tic = time.time()
        self.sim.run(tstop)
        toc = time.time() - tic
    
    def experiments(self):
        for x in ['r_ei','pconn','stim_dur','rate']:
            for y in self.param[x].keys():
                print(x, y)
                if x == 'r_ei':
                    r_ei=self.param[x][y]
                else:
                    r_ei=self.param['r_ei']['default']

                if x == 'pconn':
                    pconn = self.param[x][y]
                else:
                    pconn = self.param['pconn']['default']

                if x == 'Gexc':
                    Gexc = self.param[x][y]
                else:
                    Gexc = self.param['Gexc']['default']

                if x == 'Ginh':
                    Ginh = self.param[x][y]
                else:
                    Ginh = self.param['Ginh']['default']
                    
                if x == 'stim_dur':
                    stim_dur = self.param[x][y]
                else:
                    stim_dur = self.param['stim_dur']['default']
                
                if x == 'rate':
                    rate = self.param[x][y]
                else:
                    rate = self.param['rate']['default']

                self.create_populations(r_ei)
                self.create_connections(pconn, Gexc, Ginh, stim_dur, rate)
                self.run_simulation()
                self.save_results(self.pops, f'{saveName}-{x}-{y}.pkl')
                self.reset()
                
    def save_results(self, pops, filename):
        stateVars = {}
        for pop in pops.keys():
            print(pop)
            for recording in ['v', 'gsyn_inh', 'gsyn_exc', 'spikes']:
                print(recording)
                pops[pop].write_data(filename)
                stateVars[pop]=pops[pop].get_data()
        self.stateVarsSet.append(stateVars)
        print(stateVars.keys())
        print(stateVars)
        results = self.recover_results(stateVars)
        self.resultsSet.append(results)
        fig, ax = plt.subplots(2,1, sharex=True, figsize=(9,5))
        p0=ax[0].eventplot(results['exc', 'spikes'], color='green')
        p1=ax[1].eventplot(results['inh', 'spikes'], color='red')
        [ax[i].set_xlabel('Time (ms)', fontsize=14) for i in [0,1]]
        [ax[i].set_ylabel(feat, fontsize=14) for i, feat in enumerate(['Excitatory cells', 'Inhibitory cells'])]
        
    def recover_results(self, outputs):
        results = {}
        for key in outputs.keys(): # to extract the name of the layer, e.g., Exc, Inh, Thalamus, etc
            print(key)
            # to get voltage and conductances
            for analogsignal in outputs[key].segments[0].analogsignals:
                print(analogsignal.name)
                results[key, analogsignal.name] = analogsignal

            # to get spikes
            VAR=outputs[key].segments[0].spiketrains
            results[key, 'spikes']=[]
            for k in range(len(VAR)):
                results[key, 'spikes'].append(np.array(list(VAR[k])).T,)
        return results    
        
    def reset(self):
        self.sim.reset()

In [8]:
exp = simulations(sim, celltype, cell_params, rng, param)
exp.experiments()

2024-02-07 11:04:54 INFO: Starting execution process
2024-02-07 11:04:54 INFO: Simulating for 1000 1.0ms timesteps using a hardware timestep of 1000us


r_ei 1
750 750
4.0 51.0
0.004 0.051000000000000004
0.02


Adding Splitter selectors where appropriate
|0%                          50%                         100%|
2024-02-07 11:04:54 INFO: Time 0:00:00.011830 taken by SpynnakerSplitterSelector
Adding delay extensions as required
|0%                          50%                         100%|
2024-02-07 11:04:54 INFO: Time 0:00:00.017097 taken by DelaySupportAdder
2024-02-07 11:04:59 INFO: Time 0:00:05.574255 taken by SpallocMaxMachineGenerator
Preallocating resources for Extra Monitor support vertices
|0%                          50%                         100%|
2024-02-07 11:05:21 INFO: Time 0:00:21.102623 taken by PreAllocateResourcesForExtraMonitorSupport
Partitioning graph vertices
|0%                          50%                         100%|
Partitioning graph edges
|0%                          50%                         100%|
2024-02-07 11:05:28 INFO: Time 0:00:07.116236 taken by SpYNNakerSplitterPartitioner
Created spalloc job 389320
2024-02-07 11:05:28 INFO: Created spalloc job 38

exc
v


Getting gsyn_exc for Excitatory_Cells
|0%                          50%                         100%|
Getting gsyn_inh for Excitatory_Cells
|0%                          50%                         100%|
Getting spikes for Excitatory_Cells
|0%                          50%                         100%|
Getting v for Excitatory_Cells
|0%                          50%                         100%|
Getting gsyn_exc for Excitatory_Cells
|0%                          50%                         100%|
Getting gsyn_inh for Excitatory_Cells
|0%                          50%                         100%|
Getting spikes for Excitatory_Cells
|0%                          50%                         100%|


...

gsyn_exc
spikes


2024-02-07 12:25:29 WARNING: No Data available for Segment 6
2024-02-07 12:25:29 WARNING: No Data available for Segment 7
2024-02-07 12:25:29 WARNING: No Data available for Segment 8
2024-02-07 12:25:29 WARNING: No Data available for Segment 9
2024-02-07 12:25:29 WARNING: No Data available for Segment 10
2024-02-07 12:25:29 WARNING: No Data available for Segment 11
2024-02-07 12:25:29 WARNING: No Data available for Segment 12
2024-02-07 12:25:29 WARNING: No Data available for Segment 13
2024-02-07 12:25:29 WARNING: No Data available for Segment 14
2024-02-07 12:25:29 WARNING: No Data available for Segment 15
2024-02-07 12:25:29 WARNING: No Data available for Segment 16
2024-02-07 12:25:29 WARNING: No Data available for Segment 17
2024-02-07 12:25:29 WARNING: No Data available for Segment 18
2024-02-07 12:25:29 WARNING: No Data available for Segment 19
2024-02-07 12:25:29 WARNING: No Data available for Segment 20
Getting spikes for expoisson63
|0%                          50%           

dict_keys(['exc', 'inh', 'thalamus'])
{'exc': <neo.core.block.Block object at 0x7f133577e1f0>, 'inh': <neo.core.block.Block object at 0x7f1337fde130>, 'thalamus': <neo.core.block.Block object at 0x7f134cc52f40>}
exc
v
gsyn_exc
gsyn_inh
inh
v
gsyn_exc
gsyn_inh
thalamus


Canvas(toolbar=None)

2024-02-07 12:25:34 INFO: Caching data for segment 21
Getting spikes for Excitatory_Cells
|0%                          50%                         100%|
Getting v for Excitatory_Cells
|0%                          50%                         100%|
Getting gsyn_exc for Excitatory_Cells
|0%                          50%                         100%|
Getting gsyn_inh for Excitatory_Cells
|0%                          50%                         100%|
2024-02-07 12:25:34 INFO: Caching data for segment 21
Getting spikes for Inhibitory_Cells
|0%                          50%                         100%|
Getting v for Inhibitory_Cells
|0%                          50%                         100%|
Getting gsyn_exc for Inhibitory_Cells
|0%                          50%                         100%|
Getting gsyn_inh for Inhibitory_Cells
|0%                          50%                         100%|
2024-02-07 12:25:34 INFO: Caching data for segment 21
Getting spikes for expoisson
|0%                

In [9]:
spike_count_lists = []
for results in exp.resultsSet:
    spike_count = []
    for x in range(0,999):
        tot=0
        for ele in results['exc', 'spikes']:
            if x in ele:
                tot += 1
        spike_count.append(tot)
    spike_count_lists.append(spike_count)

In [10]:
timesteps = list(range(999)) 
firing_rates_simulation = {
    'r_ie_1': spike_count_lists[0],
    'r_ie_2': spike_count_lists[1],
    'r_ie_3': spike_count_lists[2],
    'r_ie_4': spike_count_lists[3],
    'r_ie_5': spike_count_lists[4],
    'r_ie_6': spike_count_lists[5],
    'r_ie_7': spike_count_lists[6],
    'r_ie_8': spike_count_lists[7],
}
import matplotlib.pyplot as plt

# Create the plot
plt.figure(figsize=(10, 6))  

# Plot each simulation's average firing rate over time
for param_value, rates in firing_rates_simulation.items():
    plt.plot(timesteps, rates, marker='o', linestyle='-', label=f'Param {param_value}')

# Adding labels, title, and legend
plt.xlabel('Timesteps')
plt.ylabel('Average Firing Rate')
plt.title('Average Firing Rate over Time for Varying Parameter')
plt.legend()

# Add grid for easier reading
plt.grid(True)

plt.show()
plt.savefig('exc_f.png', dpi=1500) 

plt.close()

Canvas(toolbar=None)

In [11]:
import numpy as np

# Function to calculate averages every 50 timesteps
def calculate_averages_every_50(rates):
    # Convert the list to a numpy array
    rates_array = np.array(rates)
    # Calculate the number of complete chunks of 50
    num_chunks = len(rates) // 50
    # Calculate averages for each chunk
    averages = [np.mean(rates_array[i*50:(i+1)*50]) for i in range(num_chunks)]
    return averages

# Apply the function to each simulation's firing rates
averaged_firing_rates_simulation = {param: calculate_averages_every_50(rates) for param, rates in firing_rates_simulation.items()}

# Adjusted timesteps for plotting (using the midpoint of each 50-timestep interval as the x-value)
adjusted_timesteps = [25 + i*50 for i in range(len(averaged_firing_rates_simulation['r_ie_1']))]
# Create the plot
plt.figure(figsize=(12, 7))

# Plot each simulation's averaged firing rate over time
for param_value, averages in averaged_firing_rates_simulation.items():
    plt.plot(adjusted_timesteps, averages, marker='o', linestyle='-', label=f'{param_value}')

# Adding labels, title, and legend
plt.xlabel('Timesteps (midpoint of 50-timestep interval)')
plt.ylabel('Average Firing Rate')
plt.title('Average Firing Rate Every 50 Timesteps for Varying Parameter')
plt.legend()

#Add grid 
plt.grid(True)

plt.show()

# Save the plot
plt.savefig('averaged_exc_f_50.png', dpi=500)  

# Close the plot
plt.close()


Canvas(toolbar=None)

In [12]:
spike_count_lists_inh = []
for results in exp.resultsSet:
    spike_count = []
    for x in range(0,999):
        tot=0
        for ele in results['inh', 'spikes']:
            if x in ele:
                tot += 1
        spike_count.append(tot)
    spike_count_lists_inh.append(spike_count)

In [13]:
timesteps = list(range(999)) 
firing_rates_simulation = {
    'r_ie_1': spike_count_lists_inh[0],
    'r_ie_2': spike_count_lists_inh[1],
    'r_ie_3': spike_count_lists_inh[2],
    'r_ie_4': spike_count_lists_inh[3],
    'r_ie_5': spike_count_lists_inh[4],
    'r_ie_6': spike_count_lists_inh[5],
    'r_ie_7': spike_count_lists_inh[6],
    'r_ie_8': spike_count_lists_inh[7]
}
import matplotlib.pyplot as plt

# Create the plot
plt.figure(figsize=(10, 6))  

# Plot each simulation's average firing rate over time
for param_value, rates in firing_rates_simulation.items():
    plt.plot(timesteps, rates, marker='o', linestyle='-', label=f'Param {param_value}')

# Adding labels, title, and legend
plt.xlabel('Timesteps')
plt.ylabel('Average Firing Rate')
plt.title('Average Firing Rate over Time for Varying Parameter')
plt.legend()

#Add grid for easier reading
plt.grid(True)

# Show the plot
plt.show()
plt.savefig('inh_f.png', dpi=1500) 

plt.close()

Canvas(toolbar=None)

In [14]:
import numpy as np

# Function to calculate averages every 50 timesteps
def calculate_averages_every_50(rates):
    # Convert the list to a numpy array
    rates_array = np.array(rates)
    # Calculate the number of complete chunks of 50
    num_chunks = len(rates) // 50
    # Calculate averages for each chunk
    averages = [np.mean(rates_array[i*50:(i+1)*50]) for i in range(num_chunks)]
    return averages

# Apply the function to each simulation's firing rates
averaged_firing_rates_simulation = {param: calculate_averages_every_50(rates) for param, rates in firing_rates_simulation.items()}

# Adjusted timesteps for plotting (using the midpoint of each 50-timestep interval as the x-value)
adjusted_timesteps = [25 + i*50 for i in range(len(averaged_firing_rates_simulation['r_ie_1']))]
# Create the plot with adjusted figure size
plt.figure(figsize=(12, 7))

# Plot each simulation's averaged firing rate over time
for param_value, averages in averaged_firing_rates_simulation.items():
    plt.plot(adjusted_timesteps, averages, marker='o', linestyle='-', label=f'{param_value}')

# Adding labels, title, and legend
plt.xlabel('Timesteps (midpoint of 50-timestep interval)')
plt.ylabel('Average Firing Rate')
plt.title('Average Firing Rate Every 50 Timesteps for Varying Parameter')
plt.legend()

#Add grid 
plt.grid(True)

plt.show()

# Save the plot
plt.savefig('averaged_inh_f_50.png', dpi=500)  # Adjust filename and DPI as needed

# Close the plot
plt.close()

Canvas(toolbar=None)

In [15]:
timesteps = list(range(999)) 
firing_rates_simulation = {
    'pconn_1': spike_count_lists[8],
    'pconn_2': spike_count_lists[9],
    'pconn_3': spike_count_lists[10],
    'pconn_4': spike_count_lists[11],
}
import matplotlib.pyplot as plt

# Create the plot
plt.figure(figsize=(10, 6))  

# Plot each simulation's average firing rate over time
for param_value, rates in firing_rates_simulation.items():
    plt.plot(timesteps, rates, marker='o', linestyle='-', label=f'Param {param_value}')

# Adding labels, title, and legend
plt.xlabel('Timesteps')
plt.ylabel('Average Firing Rate')
plt.title('Average Firing Rate over Time for Varying Parameter')
plt.legend()

# Optional: Add grid for easier reading
plt.grid(True)

# Show the plot
plt.show()
plt.savefig('exc_pconn.png', dpi=1500) 

plt.close()
import numpy as np

# Function to calculate averages every 50 timesteps
def calculate_averages_every_50(rates):
    # Convert the list to a numpy array for easier slicing
    rates_array = np.array(rates)
    # Calculate the number of complete chunks of 50
    num_chunks = len(rates) // 50
    # Calculate averages for each chunk
    averages = [np.mean(rates_array[i*50:(i+1)*50]) for i in range(num_chunks)]
    return averages

# Apply the function to each simulation's firing rates
averaged_firing_rates_simulation = {param: calculate_averages_every_50(rates) for param, rates in firing_rates_simulation.items()}

# Adjusted timesteps for plotting (using the midpoint of each 50-timestep interval as the x-value)
adjusted_timesteps = [25 + i*50 for i in range(len(averaged_firing_rates_simulation['pconn_1']))]
# Create the plot with adjusted figure size
plt.figure(figsize=(12, 7))

# Plot each simulation's averaged firing rate over time
for param_value, averages in averaged_firing_rates_simulation.items():
    plt.plot(adjusted_timesteps, averages, marker='o', linestyle='-', label=f'{param_value}')

# Adding labels, title, and legend
plt.xlabel('Timesteps (midpoint of 50-timestep interval)')
plt.ylabel('Average Firing Rate')
plt.title('Average Firing Rate Every 50 Timesteps for Varying Parameter')
plt.legend()

#Add grid
plt.grid(True)

# Show the plot
plt.show()

# Save the plot
plt.savefig('averaged_exc_pconn_50.png', dpi=500)  # Adjust filename and DPI as needed

# Close the plot
plt.close()

Canvas(toolbar=None)

Canvas(toolbar=None)

In [16]:
timesteps = list(range(999)) 
firing_rates_simulation = {
    'pconn_1': spike_count_lists_inh[8],
    'pconn_2': spike_count_lists_inh[9],
    'pconn_3': spike_count_lists_inh[10],
    'pconn_4': spike_count_lists_inh[11],
}
import matplotlib.pyplot as plt

# Create the plot
plt.figure(figsize=(10, 6)) 

# Plot each simulation's average firing rate over time
for param_value, rates in firing_rates_simulation.items():
    plt.plot(timesteps, rates, marker='o', linestyle='-', label=f'Param {param_value}')

# Adding labels, title, and legend
plt.xlabel('Timesteps')
plt.ylabel('Average Firing Rate')
plt.title('Average Firing Rate over Time for Varying Parameter')
plt.legend()

# Add grid
plt.grid(True)

# Show the plot
plt.show()
plt.savefig('inh_pconn.png', dpi=1500) 

plt.close()
import numpy as np

# Function to calculate averages every 50 timesteps
def calculate_averages_every_50(rates):
    # Convert the list to a numpy array 
    rates_array = np.array(rates)
    # Calculate the number of complete chunks of 50
    num_chunks = len(rates) // 50
    # Calculate averages for each chunk
    averages = [np.mean(rates_array[i*50:(i+1)*50]) for i in range(num_chunks)]
    return averages

# Apply the function to each simulation's firing rates
averaged_firing_rates_simulation = {param: calculate_averages_every_50(rates) for param, rates in firing_rates_simulation.items()}

# Adjusted timesteps for plotting (using the midpoint of each 50-timestep interval as the x-value)
adjusted_timesteps = [25 + i*50 for i in range(len(averaged_firing_rates_simulation['pconn_1']))]
# Create the plot with adjusted figure size
plt.figure(figsize=(12, 7))

# Plot each simulation's averaged firing rate over time
for param_value, averages in averaged_firing_rates_simulation.items():
    plt.plot(adjusted_timesteps, averages, marker='o', linestyle='-', label=f'{param_value}')

# Adding labels, title, and legend
plt.xlabel('Timesteps (midpoint of 50-timestep interval)')
plt.ylabel('Average Firing Rate')
plt.title('Average Firing Rate Every 50 Timesteps for Varying Parameter')
plt.legend()

#Add grid 
plt.grid(True)

# Show the plot
plt.show()

# Save the plot
plt.savefig('averaged_inh_pconn_50.png', dpi=500)  # Adjust filename and DPI as needed

# Close the plot
plt.close()

Canvas(toolbar=None)

Canvas(toolbar=None)

In [17]:
timesteps = list(range(999)) 
firing_rates_simulation = {
    'stim_dur_1': spike_count_lists[12],
    'stim_dur_2': spike_count_lists[13],
    'stim_dur_3': spike_count_lists[14],
    'stim_dur_4': spike_count_lists[15],
    'stim_dur_5': spike_count_lists[16]
}
import matplotlib.pyplot as plt

# Create the plot
plt.figure(figsize=(10, 6)) 

# Plot each simulation's average firing rate over time
for param_value, rates in firing_rates_simulation.items():
    plt.plot(timesteps, rates, marker='o', linestyle='-', label=f'Param {param_value}')

# Adding labels, title, and legend
plt.xlabel('Timesteps')
plt.ylabel('Average Firing Rate')
plt.title('Average Firing Rate over Time for Varying Parameter')
plt.legend()

#Add grid 
plt.grid(True)

# Show the plot
plt.show()
plt.savefig('exc_stim_dur.png', dpi=1500) 

plt.close()
import numpy as np

# Function to calculate averages every 50 timesteps
def calculate_averages_every_50(rates):
    # Convert the list to a numpy array
    rates_array = np.array(rates)
    # Calculate the number of complete chunks of 50
    num_chunks = len(rates) // 50
    # Calculate averages for each chunk
    averages = [np.mean(rates_array[i*50:(i+1)*50]) for i in range(num_chunks)]
    return averages

# Apply the function to each simulation's firing rates
averaged_firing_rates_simulation = {param: calculate_averages_every_50(rates) for param, rates in firing_rates_simulation.items()}

# Adjusted timesteps for plotting (using the midpoint of each 50-timestep interval as the x-value)
adjusted_timesteps = [25 + i*50 for i in range(len(averaged_firing_rates_simulation['stim_dur_1']))]
plt.figure(figsize=(12, 7))

# Plot each simulation's averaged firing rate over time
for param_value, averages in averaged_firing_rates_simulation.items():
    plt.plot(adjusted_timesteps, averages, marker='o', linestyle='-', label=f'{param_value}')

# Adding labels, title, and legend
plt.xlabel('Timesteps (midpoint of 50-timestep interval)')
plt.ylabel('Average Firing Rate')
plt.title('Average Firing Rate Every 50 Timesteps for Varying Parameter')
plt.legend()

#Add grid for 
plt.grid(True)

# Show the plot
plt.show()

# Save the plot
plt.savefig('averaged_exc_stim_dur_50.png', dpi=500)  # Adjust filename and DPI as needed

# Close the plot
plt.close()

Canvas(toolbar=None)

Canvas(toolbar=None)

In [18]:
timesteps = list(range(999)) 
firing_rates_simulation = {
    'stim_dur_1': spike_count_lists_inh[12],
    'stim_dur_2': spike_count_lists_inh[13],
    'stim_dur_3': spike_count_lists_inh[14],
    'stim_dur_4': spike_count_lists_inh[15],
    'stim_dur_5': spike_count_lists_inh[16]
}
import matplotlib.pyplot as plt

# Create the plot
plt.figure(figsize=(10, 6))  

# Plot each simulation's average firing rate over time
for param_value, rates in firing_rates_simulation.items():
    plt.plot(timesteps, rates, marker='o', linestyle='-', label=f'Param {param_value}')

# Adding labels, title, and legend
plt.xlabel('Timesteps')
plt.ylabel('Average Firing Rate')
plt.title('Average Firing Rate over Time for Varying Parameter')
plt.legend()

#Add grid
plt.grid(True)

# Show the plot
plt.show()
plt.savefig('inh_stim_dur.png', dpi=1500)  
plt.close()
import numpy as np

# Function to calculate averages every 50 timesteps
def calculate_averages_every_50(rates):
    # Convert the list to a numpy array for easier slicing
    rates_array = np.array(rates)
    # Calculate the number of complete chunks of 50
    num_chunks = len(rates) // 50
    # Calculate averages for each chunk
    averages = [np.mean(rates_array[i*50:(i+1)*50]) for i in range(num_chunks)]
    return averages

# Apply the function to each simulation's firing rates
averaged_firing_rates_simulation = {param: calculate_averages_every_50(rates) for param, rates in firing_rates_simulation.items()}

# Adjusted timesteps for plotting (using the midpoint of each 50-timestep interval as the x-value)
adjusted_timesteps = [25 + i*50 for i in range(len(averaged_firing_rates_simulation['stim_dur_1']))]

plt.figure(figsize=(12, 7))

# Plot each simulation's averaged firing rate over time
for param_value, averages in averaged_firing_rates_simulation.items():
    plt.plot(adjusted_timesteps, averages, marker='o', linestyle='-', label=f'{param_value}')

# Adding labels, title, and legend
plt.xlabel('Timesteps (midpoint of 50-timestep interval)')
plt.ylabel('Average Firing Rate')
plt.title('Average Firing Rate Every 50 Timesteps for Varying Parameter')
plt.legend()

#Add grid
plt.grid(True)

# Show the plot
plt.show()

# Save the plot
plt.savefig('averaged_inh_stim_dur_50.png', dpi=500)  # Adjust filename and DPI as needed

# Close the plot
plt.close()

Canvas(toolbar=None)

Canvas(toolbar=None)

In [19]:
timesteps = list(range(999)) 
firing_rates_simulation = {
    'rate_1': spike_count_lists[17],
    'rate_2': spike_count_lists[18],
    'rate_3': spike_count_lists[19],
    'rate_4': spike_count_lists[20],
    'rate_5': spike_count_lists[21],
}
import matplotlib.pyplot as plt

# Create the plot
plt.figure(figsize=(10, 6))  

# Plot each simulation's average firing rate over time
for param_value, rates in firing_rates_simulation.items():
    plt.plot(timesteps, rates, marker='o', linestyle='-', label=f'Param {param_value}')

# Adding labels, title, and legend
plt.xlabel('Timesteps')
plt.ylabel('Average Firing Rate')
plt.title('Average Firing Rate over Time for Varying Parameter')
plt.legend()

#Add grid 
plt.grid(True)

# Show the plot
plt.show()
plt.savefig('exc_rate.png', dpi=1500)  # Specify the path and filename, and optionally the DPI

# After saving, you might want to close the plot if you're done with it
plt.close()
import numpy as np

# Function to calculate averages every 50 timesteps
def calculate_averages_every_50(rates):
    # Convert the list to a numpy array for easier slicing
    rates_array = np.array(rates)
    # Calculate the number of complete chunks of 50
    num_chunks = len(rates) // 50
    # Calculate averages for each chunk
    averages = [np.mean(rates_array[i*50:(i+1)*50]) for i in range(num_chunks)]
    return averages

# Apply the function to each simulation's firing rates
averaged_firing_rates_simulation = {param: calculate_averages_every_50(rates) for param, rates in firing_rates_simulation.items()}

# Adjusted timesteps for plotting (using the midpoint of each 50-timestep interval as the x-value)
adjusted_timesteps = [25 + i*50 for i in range(len(averaged_firing_rates_simulation['rate_1']))]

plt.figure(figsize=(12, 7))

# Plot each simulation's averaged firing rate over time
for param_value, averages in averaged_firing_rates_simulation.items():
    plt.plot(adjusted_timesteps, averages, marker='o', linestyle='-', label=f'{param_value}')

# Adding labels, title, and legend
plt.xlabel('Timesteps (midpoint of 50-timestep interval)')
plt.ylabel('Average Firing Rate')
plt.title('Average Firing Rate Every 50 Timesteps for Varying Parameter')
plt.legend()

#Add grid 
plt.grid(True)

# Show the plot
plt.show()

# Save the plot
plt.savefig('averaged_exc_rate_50.png', dpi=500)  # Adjust filename and DPI as needed

# Close the plot
plt.close()

timesteps = list(range(999)) 
firing_rates_simulation = {
    'rate_1': spike_count_lists_inh[17],
    'rate_2': spike_count_lists_inh[18],
    'rate_3': spike_count_lists_inh[19],
    'rate_4': spike_count_lists_inh[20],
    'rate_5': spike_count_lists_inh[21]
}
import matplotlib.pyplot as plt

# Create the plot
plt.figure(figsize=(10, 6))  

# Plot each simulation's average firing rate over time
for param_value, rates in firing_rates_simulation.items():
    plt.plot(timesteps, rates, marker='o', linestyle='-', label=f'Param {param_value}')

# Adding labels, title, and legend
plt.xlabel('Timesteps')
plt.ylabel('Average Firing Rate')
plt.title('Average Firing Rate over Time for Varying Parameter')
plt.legend()

# Add grid 
plt.grid(True)

# Show the plot
plt.show()
plt.savefig('inh_rate.png', dpi=1500)  

plt.close()
import numpy as np

# Function to calculate averages every 50 timesteps
def calculate_averages_every_50(rates):
    # Convert the list to a numpy array for easier slicing
    rates_array = np.array(rates)
    # Calculate the number of complete chunks of 50
    num_chunks = len(rates) // 50
    # Calculate averages for each chunk
    averages = [np.mean(rates_array[i*50:(i+1)*50]) for i in range(num_chunks)]
    return averages

# Apply the function to each simulation's firing rates
averaged_firing_rates_simulation = {param: calculate_averages_every_50(rates) for param, rates in firing_rates_simulation.items()}

# Adjusted timesteps for plotting (using the midpoint of each 50-timestep interval as the x-value)
adjusted_timesteps = [25 + i*50 for i in range(len(averaged_firing_rates_simulation['rate_1']))]

plt.figure(figsize=(12, 7))

# Plot each simulation's averaged firing rate over time
for param_value, averages in averaged_firing_rates_simulation.items():
    plt.plot(adjusted_timesteps, averages, marker='o', linestyle='-', label=f'{param_value}')

# Adding labels, title, and legend
plt.xlabel('Timesteps (midpoint of 50-timestep interval)')
plt.ylabel('Average Firing Rate')
plt.title('Average Firing Rate Every 50 Timesteps for Varying Parameter')
plt.legend()

#Add grid
plt.grid(True)

# Show the plot
plt.show()

# Save the plot
plt.savefig('averaged_inh_rate_50.png', dpi=500)  # Adjust filename and DPI as needed

# Close the plot
plt.close()

Canvas(toolbar=None)

Canvas(toolbar=None)

Canvas(toolbar=None)

Canvas(toolbar=None)